<div style="
    border: 3px solid #4CAF50;  /* Green border */
    background: linear-gradient(90deg, #ffcccb, #ffe066, #b3ffb3);  /* Colorful gradient */
    padding: 20px;
    border-radius: 15px;
    text-align: center;
    font-family: Arial, sans-serif;
">
    <h2 style="
        font-weight: bold;
        font-size: 28px;
        color: #1a1a1a;
        text-shadow: 2px 2px 4px #888888;
    ">
        Congratulation to all <span style="color: #FF4500;">1st Batch</span> 
        <span style="color: #4169E1;">Mella-Python-Data-Analytics-Group</span>
    </h2>
</div>

In [1]:
###############################################################################################
import Mella_python_data_analytics as mella # This is the New Module that developed by the first Batch students
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
###############################################################################################
conn_str = "" # Create Your Own connection String
folder_path = r"" # Put your own directory path
###############################################################################################
plots_dir = os.path.join(folder_path, "Plots")
os.makedirs(plots_dir, exist_ok=True)
###############################################################################################
columns_to_keep = [
    'STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'PRCP',
    'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES'
]
###############################################################################################
dataframes_dict = mella.read_individual_csv_files(folder_path)
csv_files = list(dataframes_dict.keys())
print(f"Found {len(csv_files)} CSV files to process")
###############################################################################################
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    print(f"\n{'='*60}")
    print(f"PROCESSING FILE: {csv_file}")
    print(f"{'='*60}")
###############################################################################################
    df = pd.read_csv(file_path)

###############################################################################################    
    available_columns = [col for col in columns_to_keep if col in df.columns]
    missing_columns = [col for col in columns_to_keep if col not in df.columns]
    if missing_columns:
        print(f"Missing columns: {missing_columns}")
###############################################################################################   
    df_processed = df[available_columns].copy()
    df_processed = mella.handle_missing_values(df_processed, method="fill")
###############################################################################################    

    if 'DATE' in df_processed.columns:
        df_processed = mella.convert_datetime_column(df_processed, "DATE", format="%Y-%m-%d")
    
    # Remove duplicates
    duplicate_cols = []
    if 'STATION' in df_processed.columns and 'DATE' in df_processed.columns:
        duplicate_cols = ["STATION", "DATE"]
    elif 'STATION' in df_processed.columns:
        duplicate_cols = ["STATION"]
    
    if duplicate_cols:
        df_processed = mella.drop_duplicates(df_processed, subset=duplicate_cols, keep='first')
    else:
        df_processed = mella.drop_duplicates(df_processed, keep='first')
    
    print(f"Shape after cleaning: {df_processed.shape}")
    
    # Convert numeric columns
    numeric_cols_to_convert = ['PRCP', 'SNOW', 'LATITUDE', 'LONGITUDE', 'ELEVATION']
    for col in numeric_cols_to_convert:
        if col in df_processed.columns:
            df_processed = mella.convert_to_numeric(df_processed, col)
# Don't worry much about the simulations block **********************************************************   
    # Plotings, subplots

    fig = plt.figure(figsize=(15, 10))
    fig.suptitle(f"Analysis of {csv_file}", fontsize=16, fontweight='bold')

    ax1 = plt.subplot(2, 2, 1)
    available_count = len(available_columns)
    missing_count = len(missing_columns)
    ax1.pie([available_count, missing_count], 
            labels=['Available', 'Missing'], autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    ax1.set_title('Requested Columns Availability')
    
    # Plot 2: Missing values in kept columns
    ax2 = plt.subplot(2, 2, 2)
    missing_values = df_processed.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    if len(missing_values) > 0:
        ax2.bar(missing_values.index, missing_values.values, color='orange')
        ax2.set_title('Missing Values in Kept Columns')
        ax2.tick_params(axis='x', rotation=45)
    else:
        ax2.text(0.5, 0.5, 'No missing values', ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Missing Values')
    
    # Plot 3: Precipitation distribution (if available)
    ax3 = plt.subplot(2, 2, 3)
    if 'PRCP' in df_processed.columns:
        prcp_data = df_processed['PRCP'].dropna()
        if len(prcp_data) > 0:
            ax3.hist(prcp_data, bins=20, alpha=0.7, color='skyblue')
            ax3.set_title('Precipitation (PRCP) Distribution')
            ax3.set_xlabel('Precipitation')
            ax3.set_ylabel('Frequency')
        else:
            ax3.text(0.5, 0.5, 'No PRCP data', ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('Precipitation Data')
    else:
        ax3.text(0.5, 0.5, 'PRCP column not available', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Precipitation Data')
    
    # Plot 4: File summary
    ax4 = plt.subplot(2, 2, 4)
    ax4.axis('off')
    summary_text = f"""
    File: {csv_file}
    Original Records: {len(df)}
    Cleaned Records: {len(df_processed)}
    Kept Columns: {len(available_columns)}
    Missing Columns: {len(missing_columns)}
    Stations: {df_processed['STATION'].nunique() if 'STATION' in df_processed.columns else 'N/A'}
    """
    ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes, fontfamily='monospace',
             verticalalignment='top', fontsize=10)
    ax4.set_title('File Summary')
    
    plt.tight_layout()
    
    # Save the summary plot
    plot_filename = os.path.splitext(csv_file)[0] + '_summary.png'
    plot_path = os.path.join(plots_dir, plot_filename)
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved summary plot: {plot_filename}")
    # Create individual plots for numeric columns
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        try:
            plt.figure(figsize=(10, 6))
            plt.hist(df_processed[col].dropna(), bins=20, alpha=0.7, color='lightseagreen')
            plt.title(f'Distribution of {col} - {csv_file}')
            plt.xlabel(col)
            plt.ylabel('Frequency')
            plt.grid(True, alpha=0.3)
            
            # Save individual plot
            col_plot_filename = os.path.splitext(csv_file)[0] + f'_{col}_distribution.png'
            col_plot_path = os.path.join(plots_dir, col_plot_filename)
            plt.savefig(col_plot_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"  Saved {col} distribution plot")
            
        except Exception as e:
            print(f"  Error creating plot for {col}: {e}")
# Don't worry much about the simulations block ********************************************************** 

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%   This the Integration Block between SQL and Python   %%%%%%%%%%%%%%
    print(f"\nExporting {csv_file} to SQL Server...")
    try:
        # You can give a proper short names for your tables to be export to SQL
        table_name = os.path.splitext(csv_file)[0]
        table_name = ''.join(c for c in table_name if c.isalnum() or c in ['_', ' '])
        table_name = table_name.replace(' ', '_') 
        table_name = f"Weather_{table_name}"  

        if len(table_name) > 100:
            table_name = table_name[:100]
        
        mella.export_to_mssql(df_processed, table_name, conn_str)
        print(f"Successfully exported to table: {table_name}")

        export_summary = pd.DataFrame({
            'filename': [csv_file],
            'table_name': [table_name],
            'original_rows': [len(df)],
            'cleaned_rows': [len(df_processed)],
            'kept_columns': [len(available_columns)],
            'missing_columns': [len(missing_columns)],
            'export_time': [pd.Timestamp.now()]
        })
        
        export_log_path = os.path.join(folder_path, "export_log.csv")
        if os.path.exists(export_log_path):
            export_log = pd.read_csv(export_log_path)
            export_log = pd.concat([export_log, export_summary], ignore_index=True)
        else:
            export_log = export_summary
        
        export_log.to_csv(export_log_path, index=False)
        print(f"Export logged to: {export_log_path}")
        
    except Exception as e:
        print(f"Error exporting {csv_file} to database: {e}")
    
    print(f"\nCompleted processing: {csv_file}")

#    You can Uncomment the commented lines here below, if you want to seed the direct outputs
#     print(f"{'='*60}")
# print(f"\n{'='*60}")
# print("PROCESSING COMPLETE!")
# print(f"{'='*60}")
# print(f"Total files processed: {len(csv_files)}")
# print(f"Plots saved in: {plots_dir}")
# print(f"All data exported to SQL Server database: Stu_Database")
# print(f"Each file exported to separate table with 'Weather_' prefix")
# Show column availability summary

print(f"\nColumn Availability Summary:")
column_availability = {}
for col in columns_to_keep:
    column_availability[col] = sum(1 for csv_file in csv_files if col in pd.read_csv(os.path.join(folder_path, csv_file)).columns)

availability_df = pd.DataFrame({
    'Column': columns_to_keep,
    'Available_In_Files': [column_availability[col] for col in columns_to_keep],
    'Availability_Percentage': [f"{(column_availability[col] / len(csv_files)) * 100:.1f}%" for col in columns_to_keep]
})

print(availability_df.to_string(index=False))
# GOOD LUCK Mella-Python Group, Thank you

C:\Aru_New_Future\Mella-Technollogy-LLC-Python-Course-main\Mella-Technollogy-LLC-Python-Course-main\Week_8\Mella_python_data_analytics.py:41: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Aru_New_Future\Mella-Technollogy-LLC-Python-Course-main\Mella-Technollogy-LLC-Python-Course-main\Week_8\Mella_python_data_analytics.py:41: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,105,115,119,121,123,125,127,129,131,133,135,137,139,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Aru_New_Future\Mella-Technollogy-LLC-Python-Course-main\Mella-Tec

Found 3 CSV files to process

PROCESSING FILE: Aweather.csv


C:\Users\abreh\AppData\Local\Temp\ipykernel_27884\3931124796.py:30: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Original shape: (45841, 138)
Original columns: ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'ACMH', 'ACMH_ATTRIBUTES', 'ACSH', 'ACSH_ATTRIBUTES', 'ADPT', 'ADPT_ATTRIBUTES', 'ASLP', 'ASLP_ATTRIBUTES', 'ASTP', 'ASTP_ATTRIBUTES', 'AWBT', 'AWBT_ATTRIBUTES', 'AWND', 'AWND_ATTRIBUTES', 'DAEV', 'DAEV_ATTRIBUTES', 'DAPR', 'DAPR_ATTRIBUTES', 'DAWM', 'DAWM_ATTRIBUTES', 'EVAP', 'EVAP_ATTRIBUTES', 'FMTM', 'FMTM_ATTRIBUTES', 'FRGB', 'FRGB_ATTRIBUTES', 'FRGT', 'FRGT_ATTRIBUTES', 'FRTH', 'FRTH_ATTRIBUTES', 'GAHT', 'GAHT_ATTRIBUTES', 'MDEV', 'MDEV_ATTRIBUTES', 'MDPR', 'MDPR_ATTRIBUTES', 'MDWM', 'MDWM_ATTRIBUTES', 'PGTM', 'PGTM_ATTRIBUTES', 'PSUN', 'PSUN_ATTRIBUTES', 'RHAV', 'RHAV_ATTRIBUTES', 'RHMN', 'RHMN_ATTRIBUTES', 'RHMX', 'RHMX_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WDF1', 'WDF1_ATTRIBUTES', 'WDF2', 'WDF2_ATTRIBUTE

C:\Users\abreh\AppData\Local\Temp\ipykernel_27884\3931124796.py:30: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,105,115,119,121,123,125,127,129,131,133,135,137,139,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


...

C:\Users\abreh\AppData\Local\Temp\ipykernel_27884\3931124796.py:30: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,101,107,109,111,115,117,119,121,123,125,127,129,131,133,135,137,139,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Original shape: (31552, 144)
Original columns: ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'ACSH', 'ACSH_ATTRIBUTES', 'ADPT', 'ADPT_ATTRIBUTES', 'ASLP', 'ASLP_ATTRIBUTES', 'ASTP', 'ASTP_ATTRIBUTES', 'AWBT', 'AWBT_ATTRIBUTES', 'AWND', 'AWND_ATTRIBUTES', 'DAEV', 'DAEV_ATTRIBUTES', 'DAWM', 'DAWM_ATTRIBUTES', 'EVAP', 'EVAP_ATTRIBUTES', 'FMTM', 'FMTM_ATTRIBUTES', 'FRGB', 'FRGB_ATTRIBUTES', 'FRGT', 'FRGT_ATTRIBUTES', 'FRTH', 'FRTH_ATTRIBUTES', 'GAHT', 'GAHT_ATTRIBUTES', 'MDEV', 'MDEV_ATTRIBUTES', 'MDWM', 'MDWM_ATTRIBUTES', 'MNPN', 'MNPN_ATTRIBUTES', 'MXPN', 'MXPN_ATTRIBUTES', 'PGTM', 'PGTM_ATTRIBUTES', 'PSUN', 'PSUN_ATTRIBUTES', 'RHAV', 'RHAV_ATTRIBUTES', 'RHMN', 'RHMN_ATTRIBUTES', 'RHMX', 'RHMX_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES', 'TOBS', 'TOBS_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WDF1', 'WDF1_ATTRIBUTES', 'WDF2', 'WDF2_ATTRIBUTE

C:\Users\abreh\AppData\Local\Temp\ipykernel_27884\3931124796.py:224: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137) have mixed types. Specify dtype option on import or set low_memory=False.
  column_availability[col] = sum(1 for csv_file in csv_files if col in pd.read_csv(os.path.join(folder_path, csv_file)).columns)
C:\Users\abreh\AppData\Local\Temp\ipykernel_27884\3931124796.py:224: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,105,115,119,121,123,125,127,129,131,133,135,137,139,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  column_availability[col] = sum(1 for csv_file in csv_files if col in pd.read_csv(os.path.join(folder_path, csv_file)).columns)
C:\Users\abreh\A

         Column  Available_In_Files Availability_Percentage
        STATION                   3                  100.0%
           DATE                   3                  100.0%
       LATITUDE                   3                  100.0%
      LONGITUDE                   3                  100.0%
      ELEVATION                   3                  100.0%
           NAME                   3                  100.0%
           PRCP                   3                  100.0%
PRCP_ATTRIBUTES                   3                  100.0%
           SNOW                   3                  100.0%
SNOW_ATTRIBUTES                   3                  100.0%


C:\Users\abreh\AppData\Local\Temp\ipykernel_27884\3931124796.py:224: DtypeWarning: Columns (17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,101,107,109,111,115,117,119,121,123,125,127,129,131,133,135,137,139,141,143) have mixed types. Specify dtype option on import or set low_memory=False.
  column_availability[col] = sum(1 for csv_file in csv_files if col in pd.read_csv(os.path.join(folder_path, csv_file)).columns)


In [2]:
    print(f"\n{'='*80}")
    print(f"PROCESSING FILE: {4}")
    print(f"{'='*60}")


PROCESSING FILE: 4
